# Historical Data and Sentiment Analysis Models

The models in this notebook are trained only on the historical data obtained from CMC, sentiment data obtained from Reddit and public interest data from google trends. 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU, Conv1D, MaxPooling1D, Flatten

warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

In [2]:
data=pd.read_csv(r'C:\Users\madha\Desktop\Dissertation\Data\Master Data\Feature Engineering\FeatureEngineered_ETH.csv')

In [3]:
data.tail()

,date,name,open,high,low,close,volume,marketCap,daily_weighted_sentiment,comment_volume,...,boll,boll_ub,boll_lb,boll_width,kdjk,kdjd,kdj_signal,macd,macds,macdh
345,2025-07-10,2781,2770.737301,2995.152156,2757.266597,2954.845218,3.392920e+10,3.566962e+11,-0.205187,42.0,...,2506.585618,2820.536068,2192.635167,627.900901,85.519353,77.963274,0.0,66.939518,22.230831,44.708688
346,2025-07-11,2781,2954.832779,3038.141150,2916.956467,2957.886198,3.622656e+10,3.571174e+11,0.313511,113.0,...,2539.455146,2897.159588,2181.750703,715.408885,85.589425,80.505324,0.0,88.561261,35.496917,53.064344
347,2025-07-12,2781,2958.333774,2979.779932,2907.193557,2942.911663,1.631720e+10,3.552472e+11,-0.004126,55.0,...,2575.190071,2944.582205,2205.797938,738.784266,84.748580,81.919743,0.0,103.297578,49.057049,54.240529
348,2025-07-13,2781,2942.853579,3016.393465,2938.736541,2973.358989,1.736175e+10,3.589131e+11,-0.129803,74.0,...,2602.766776,3004.852972,2200.680580,804.172392,85.900037,83.246507,0.0,116.094792,62.464598,53.630194
349,2025-07-14,2781,2973.225148,3079.985511,2965.323790,3013.350859,3.634929e+10,3.637496e+11,0.040624,44.0,...,2631.033876,3065.495895,2196.571858,868.924037,86.734556,84.409190,0.0,127.988316,75.569341,52.418975


### Preprocessing

We will only keep OHLCV data, market cap and the target feature from this set.

In [4]:
data.columns

Index(['date', 'name', 'open', 'high', 'low', 'close', 'volume', 'marketCap',
       'daily_weighted_sentiment', 'comment_volume', 'ETH_trends', 'gold_spot',
       'gspc_spot', 'ndx_spot', 'price_change', 'target', 'percent_change_24h',
       'percent_change_3d', 'percent_change_7d', 'percent_change_14d', 'ma_7d',
       'price_vs_ma7d', 'ma_14d', 'price_vs_ma14d', 'close_7_ema',
       'close_14_ema', 'dx', 'adx', 'adxr', 'rsi_7', 'rsi', 'stochrsi', 'atr',
       'mfi', 'boll', 'boll_ub', 'boll_lb', 'boll_width', 'kdjk', 'kdjd',
       'kdj_signal', 'macd', 'macds', 'macdh'],
      dtype='object')

In [5]:
cols=['date','open','high','low','close','volume','marketCap','target','daily_weighted_sentiment', 'comment_volume', 'ETH_trends']
df = data[cols]

In [6]:
df.head()

,date,open,high,low,close,volume,marketCap,target,daily_weighted_sentiment,comment_volume,ETH_trends
0,2024-07-30,3320.635191,3365.322564,3235.759968,3278.667793,1.404577e+10,3.942544e+11,-1.0,0.217507,18.0,53.0
1,2024-07-31,3278.686596,3347.636590,3216.071707,3231.295707,1.613538e+10,3.885560e+11,-1.0,0.935494,1.0,53.0
2,2024-08-01,3231.248666,3241.777327,3078.543878,3201.564447,2.021764e+10,3.849965e+11,-1.0,-0.002735,78.0,53.0
3,2024-08-02,3201.599383,3214.527062,2965.733959,2986.013153,2.140024e+10,3.590700e+11,-1.0,0.000000,2.0,53.0
4,2024-08-03,2985.950879,3015.296052,2861.177554,2903.385703,1.784409e+10,3.491114e+11,-1.0,-0.253532,41.0,53.0


In [7]:
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', inplace=True)
df['day_of_week'] = df['date'].dt.dayofweek

In [8]:
df.head(10)

,date,open,high,low,close,volume,marketCap,target,daily_weighted_sentiment,comment_volume,ETH_trends,day_of_week
0,2024-07-30,3320.635191,3365.322564,3235.759968,3278.667793,1.404577e+10,3.942544e+11,-1.0,0.217507,18.0,53.0,1
1,2024-07-31,3278.686596,3347.636590,3216.071707,3231.295707,1.613538e+10,3.885560e+11,-1.0,0.935494,1.0,53.0,2
2,2024-08-01,3231.248666,3241.777327,3078.543878,3201.564447,2.021764e+10,3.849965e+11,-1.0,-0.002735,78.0,53.0,3
3,2024-08-02,3201.599383,3214.527062,2965.733959,2986.013153,2.140024e+10,3.590700e+11,-1.0,0.000000,2.0,53.0,4
4,2024-08-03,2985.950879,3015.296052,2861.177554,2903.385703,1.784409e+10,3.491114e+11,-1.0,-0.253532,41.0,53.0,5
5,2024-08-04,2903.088632,2931.471751,2639.566726,2686.398861,2.113960e+10,3.230151e+11,-1.0,-0.296747,44.0,72.0,6
6,2024-08-05,2686.027498,2695.886656,2122.546236,2417.206184,6.766813e+10,2.906999e+11,1.0,-0.025265,20.0,72.0,0
7,2024-08-06,2417.269748,2553.581079,2416.527009,2458.723870,2.604200e+10,2.957106e+11,-1.0,0.093450,31.0,72.0,1
8,2024-08-07,2458.988906,2551.560686,2312.168693,2336.589375,2.426422e+10,2.810354e+11,1.0,0.127970,48.0,72.0,2
9,2024-08-08,2336.914969,2721.954498,2322.529498,2683.352798,2.346829e+10,3.227475e+11,-1.0,0.097882,48.0,72.0,3


In [9]:
features = df[['day_of_week','open','high','low','close','volume','marketCap','daily_weighted_sentiment', 'comment_volume', 'ETH_trends']]
target = df['target']
target = target.replace(-1, 0)

In [10]:
features

,day_of_week,open,high,low,close,volume,marketCap,daily_weighted_sentiment,comment_volume,ETH_trends
0,1,3320.635191,3365.322564,3235.759968,3278.667793,1.404577e+10,3.942544e+11,0.217507,18.0,53.0
1,2,3278.686596,3347.636590,3216.071707,3231.295707,1.613538e+10,3.885560e+11,0.935494,1.0,53.0
2,3,3231.248666,3241.777327,3078.543878,3201.564447,2.021764e+10,3.849965e+11,-0.002735,78.0,53.0
3,4,3201.599383,3214.527062,2965.733959,2986.013153,2.140024e+10,3.590700e+11,0.000000,2.0,53.0
4,5,2985.950879,3015.296052,2861.177554,2903.385703,1.784409e+10,3.491114e+11,-0.253532,41.0,53.0
...,...,...,...,...,...,...,...,...,...,...
345,3,2770.737301,2995.152156,2757.266597,2954.845218,3.392920e+10,3.566962e+11,-0.205187,42.0,62.0
346,4,2954.832779,3038.141150,2916.956467,2957.886198,3.622656e+10,3.571174e+11,0.313511,113.0,62.0
347,5,2958.333774,2979.779932,2907.193557,2942.911663,1.631720e+10,3.552472e+11,-0.004126,55.0,62.0
348,6,2942.853579,3016.393465,2938.736541,2973.358989,1.736175e+10,3.589131e+11,-0.129803,74.0,100.0


In [11]:
split_point = int(len(df) * 0.85)
X_test_set = features[split_point:] 
y_test_set = target[split_point:]   
actuals = y_test_set.tolist()

In [12]:
window_sizes = [int(len(df) * 0.85), 150, 100, 60, 30, 20]

Creating a sequence function for sequence models.

In [13]:
def create_sequences(features, target, time_steps=10):
    Xs, ys = [], []
    for i in range(len(features) - time_steps):
        Xs.append(features[i:(i + time_steps)])
        ys.append(target[i + time_steps])
    return np.array(Xs), np.array(ys)

TIME_STEPS = 10 

## Models

The models are tested against windows that include the following sizes: 
- 85% of the dataset (297 days)
- 150 days 
- 100 days
- 60 days
- 30 days
- 20 days

### Logistic Regression

In [14]:
for window_size in window_sizes:
    predictions = []
    
    for i in range(len(X_test_set)):
        end_index = split_point + i
        start_index = end_index - window_size
            
        X_train = features.iloc[start_index:end_index]
        y_train = target.iloc[start_index:end_index]

        # This takes one day's data at one time
        X_test = X_test_set.iloc[[i]] 

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        model = LogisticRegression()
        model.fit(X_train_scaled, y_train)
        prediction = model.predict(X_test_scaled)[0]
        predictions.append(prediction)


    print(f"\nEvaluation for Logistic Regression with Window Size: {window_size}")
    current_actuals = actuals[len(actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for Logistic Regression with Window Size: 297
Accuracy: 0.5094
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.44      0.48      0.46        23
      UP (1)       0.57      0.53      0.55        30

    accuracy                           0.51        53
   macro avg       0.51      0.51      0.51        53
weighted avg       0.51      0.51      0.51        53

--------------------------------------------------

Evaluation for Logistic Regression with Window Size: 150
Accuracy: 0.4906
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.44      0.61      0.51        23
      UP (1)       0.57      0.40      0.47        30

    accuracy                           0.49        53
   macro avg       0.50      0.50      0.49        53
weighted avg       0.51      0.49      0.49        53

--------------------------------------------------

Evaluation for Logistic Regression with Window 

### Random Forest

In [15]:
for window_size in window_sizes:
    predictions = []
    
    for i in range(len(X_test_set)):
        end_index = split_point + i
        start_index = end_index - window_size
            
        X_train = features.iloc[start_index:end_index]
        y_train = target.iloc[start_index:end_index]
        X_test = X_test_set.iloc[[i]]

        model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        model.fit(X_train, y_train) 
        prediction = model.predict(X_test)[0]
        predictions.append(prediction)

    print(f"\nEvaluation for Random Forest with Window Size: {window_size}")
    current_actuals = actuals[len(actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for Random Forest with Window Size: 297
Accuracy: 0.3962
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.30      0.30      0.30        23
      UP (1)       0.47      0.47      0.47        30

    accuracy                           0.40        53
   macro avg       0.39      0.39      0.39        53
weighted avg       0.40      0.40      0.40        53

--------------------------------------------------

Evaluation for Random Forest with Window Size: 150
Accuracy: 0.3396
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.31      0.43      0.36        23
      UP (1)       0.38      0.27      0.31        30

    accuracy                           0.34        53
   macro avg       0.35      0.35      0.34        53
weighted avg       0.35      0.34      0.34        53

--------------------------------------------------

Evaluation for Random Forest with Window Size: 100
Accuracy

### XG Boost

In [16]:
for window_size in window_sizes:
    predictions = []
    
    for i in range(len(X_test_set)):
        end_index = split_point + i
        start_index = end_index - window_size
            
        X_train = features.iloc[start_index:end_index]
        y_train = target.iloc[start_index:end_index]
        X_test = X_test_set.iloc[[i]]

        model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
        model.fit(X_train, y_train) 
        prediction = model.predict(X_test)[0]
        predictions.append(prediction)

    print(f"\nEvaluation for XGB with Window Size: {window_size}")
    current_actuals = actuals[len(actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for XGB with Window Size: 297
Accuracy: 0.3019
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.21      0.22      0.21        23
      UP (1)       0.38      0.37      0.37        30

    accuracy                           0.30        53
   macro avg       0.29      0.29      0.29        53
weighted avg       0.31      0.30      0.30        53

--------------------------------------------------

Evaluation for XGB with Window Size: 150
Accuracy: 0.3396
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.27      0.30      0.29        23
      UP (1)       0.41      0.37      0.39        30

    accuracy                           0.34        53
   macro avg       0.34      0.34      0.34        53
weighted avg       0.35      0.34      0.34        53

--------------------------------------------------

Evaluation for XGB with Window Size: 100
Accuracy: 0.3585
Classification Report

### MLP Classifier

In [17]:
for window_size in window_sizes:
    predictions = []
    
    for i in range(len(X_test_set)):
        end_index = split_point + i
        start_index = end_index - window_size
            
        X_train = features.iloc[start_index:end_index]
        y_train = target.iloc[start_index:end_index]
        X_test = X_test_set.iloc[[i]]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        model = Sequential([
            Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
            Dense(16, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy')
        model.fit(X_train_scaled, y_train, epochs=10, batch_size=16, verbose=0)
    
        prediction = (model.predict(X_test_scaled, verbose=0) > 0.5).astype("int32")[0][0]
        predictions.append(prediction)
        
    print(f"\nEvaluation for MLP with Window Size: {window_size}")
    current_actuals = actuals[len(actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for MLP with Window Size: 297
Accuracy: 0.4528
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.41      0.57      0.47        23
      UP (1)       0.52      0.37      0.43        30

    accuracy                           0.45        53
   macro avg       0.47      0.47      0.45        53
weighted avg       0.47      0.45      0.45        53

--------------------------------------------------

Evaluation for MLP with Window Size: 150
Accuracy: 0.5094
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.45      0.65      0.54        23
      UP (1)       0.60      0.40      0.48        30

    accuracy                           0.51        53
   macro avg       0.53      0.53      0.51        53
weighted avg       0.54      0.51      0.50        53

--------------------------------------------------

Evaluation for MLP with Window Size: 100
Accuracy: 0.4340
Classification Report

### LSTM

In [18]:
X_test_seq = pd.concat([features.iloc[split_point - TIME_STEPS:split_point], X_test_set])
y_test_seq = pd.concat([target.iloc[split_point - TIME_STEPS:split_point], y_test_set])

In [19]:
for window_size in window_sizes:
    predictions = []
    num_predictions_possible = len(X_test_seq) - TIME_STEPS
    loop_actuals = y_test_seq[TIME_STEPS:].tolist()

    for i in range(num_predictions_possible):
        end_index = split_point + i + TIME_STEPS
        start_index = end_index - window_size - TIME_STEPS

        window_features = features.iloc[start_index:end_index]
        window_target = target.iloc[start_index:end_index]

        scaler = StandardScaler()
        window_features_scaled = scaler.fit_transform(window_features)
        
        X_seq, y_seq = create_sequences(window_features_scaled, window_target.values, TIME_STEPS)
        X_train, y_train = X_seq[:-1], y_seq[:-1]
        X_test = X_seq[-1:]
        
        model = Sequential([
            LSTM(50, activation='relu', input_shape=(TIME_STEPS, X_train.shape[2])),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy')
        model.fit(X_train, y_train, epochs=10, verbose=0)
        
        prediction = (model.predict(X_test, verbose=0) > 0.5).astype("int32")[0][0]
        predictions.append(prediction)

    print(f"\nEvaluation for LSTM with Window Size: {window_size}")
    current_actuals = loop_actuals[len(loop_actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for LSTM with Window Size: 297
Accuracy: 0.4906
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.45      0.78      0.57        23
      UP (1)       0.62      0.27      0.37        30

    accuracy                           0.49        53
   macro avg       0.53      0.52      0.47        53
weighted avg       0.54      0.49      0.46        53

--------------------------------------------------

Evaluation for LSTM with Window Size: 150
Accuracy: 0.5283
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.48      0.87      0.62        23
      UP (1)       0.73      0.27      0.39        30

    accuracy                           0.53        53
   macro avg       0.60      0.57      0.50        53
weighted avg       0.62      0.53      0.49        53

--------------------------------------------------

Evaluation for LSTM with Window Size: 100
Accuracy: 0.4717
Classification Rep

### GRU

In [20]:
for window_size in window_sizes:
    predictions = []
    num_predictions_possible = len(X_test_seq) - TIME_STEPS
    loop_actuals = y_test_seq[TIME_STEPS:].tolist()

    for i in range(num_predictions_possible):
        end_index = split_point + i + TIME_STEPS
        start_index = end_index - window_size - TIME_STEPS

        window_features = features.iloc[start_index:end_index]
        window_target = target.iloc[start_index:end_index]

        scaler = StandardScaler()
        window_features_scaled = scaler.fit_transform(window_features)
        
        X_seq, y_seq = create_sequences(window_features_scaled, window_target.values, TIME_STEPS)
        X_train, y_train = X_seq[:-1], y_seq[:-1]
        X_test = X_seq[-1:]

        model = Sequential([
            GRU(50, activation='relu', input_shape=(TIME_STEPS, X_train.shape[2])),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy')
        model.fit(X_train, y_train, epochs=10, verbose=0)
        
        prediction = (model.predict(X_test, verbose=0) > 0.5).astype("int32")[0][0]
        predictions.append(prediction)

    print(f"\nEvaluation for GRU with Window Size: {window_size}")
    current_actuals = loop_actuals[len(loop_actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for GRU with Window Size: 297
Accuracy: 0.4717
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.43      0.65      0.52        23
      UP (1)       0.56      0.33      0.42        30

    accuracy                           0.47        53
   macro avg       0.49      0.49      0.47        53
weighted avg       0.50      0.47      0.46        53

--------------------------------------------------

Evaluation for GRU with Window Size: 150
Accuracy: 0.6226
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.54      0.87      0.67        23
      UP (1)       0.81      0.43      0.57        30

    accuracy                           0.62        53
   macro avg       0.68      0.65      0.62        53
weighted avg       0.69      0.62      0.61        53

--------------------------------------------------

Evaluation for GRU with Window Size: 100
Accuracy: 0.5660
Classification Report

### 1D-CNN

In [21]:
for window_size in window_sizes:
    predictions = []
    num_predictions_possible = len(X_test_seq) - TIME_STEPS
    loop_actuals = y_test_seq[TIME_STEPS:].tolist()

    for i in range(num_predictions_possible):
        end_index = split_point + i + TIME_STEPS
        start_index = end_index - window_size - TIME_STEPS

        window_features = features.iloc[start_index:end_index]
        window_target = target.iloc[start_index:end_index]

        scaler = StandardScaler()
        window_features_scaled = scaler.fit_transform(window_features)
        
        X_seq, y_seq = create_sequences(window_features_scaled, window_target.values, TIME_STEPS)
        X_train, y_train = X_seq[:-1], y_seq[:-1]
        X_test = X_seq[-1:]
        

        model = Sequential([
            Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(TIME_STEPS, X_train.shape[2])),
            MaxPooling1D(pool_size=2),
            Flatten(),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy')
        model.fit(X_train, y_train, epochs=10, verbose=0)
        
        prediction = (model.predict(X_test, verbose=0) > 0.5).astype("int32")[0][0]
        predictions.append(prediction)

    print(f"\nEvaluation for 1D-CNN with Window Size: {window_size}")
    current_actuals = loop_actuals[len(loop_actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for 1D-CNN with Window Size: 297
Accuracy: 0.5660
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.50      0.65      0.57        23
      UP (1)       0.65      0.50      0.57        30

    accuracy                           0.57        53
   macro avg       0.58      0.58      0.57        53
weighted avg       0.59      0.57      0.57        53

--------------------------------------------------

Evaluation for 1D-CNN with Window Size: 150
Accuracy: 0.6226
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.56      0.61      0.58        23
      UP (1)       0.68      0.63      0.66        30

    accuracy                           0.62        53
   macro avg       0.62      0.62      0.62        53
weighted avg       0.63      0.62      0.62        53

--------------------------------------------------

Evaluation for 1D-CNN with Window Size: 100
Accuracy: 0.5094
Classificati

### Ensemble: 1D-CNN and LSTM

In [22]:
for window_size in window_sizes:
    predictions = []
    num_predictions_possible = len(X_test_seq) - TIME_STEPS
    loop_actuals = y_test_seq[TIME_STEPS:].tolist()

    for i in range(num_predictions_possible):
        end_index = split_point + i + TIME_STEPS
        start_index = end_index - window_size - TIME_STEPS

        window_features = features.iloc[start_index:end_index]
        window_target = target.iloc[start_index:end_index]

        scaler = StandardScaler()
        window_features_scaled = scaler.fit_transform(window_features)
        
        X_seq, y_seq = create_sequences(window_features_scaled, window_target.values, TIME_STEPS)
        X_train, y_train = X_seq[:-1], y_seq[:-1]
        X_test = X_seq[-1:]
        

        model = Sequential([
            Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(TIME_STEPS, X_train.shape[2])),
            MaxPooling1D(pool_size=2),
            LSTM(50, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy')
        model.fit(X_train, y_train, epochs=10, verbose=0)
        
        prediction = (model.predict(X_test, verbose=0) > 0.5).astype("int32")[0][0]
        predictions.append(prediction)

    print(f"\nEvaluation for 1D-CNN-LSTM with Window Size: {window_size}")
    current_actuals = loop_actuals[len(loop_actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for 1D-CNN-LSTM with Window Size: 297
Accuracy: 0.4717
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.43      0.70      0.53        23
      UP (1)       0.56      0.30      0.39        30

    accuracy                           0.47        53
   macro avg       0.50      0.50      0.46        53
weighted avg       0.51      0.47      0.45        53

--------------------------------------------------

Evaluation for 1D-CNN-LSTM with Window Size: 150
Accuracy: 0.4151
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.40      0.70      0.51        23
      UP (1)       0.46      0.20      0.28        30

    accuracy                           0.42        53
   macro avg       0.43      0.45      0.39        53
weighted avg       0.43      0.42      0.38        53

--------------------------------------------------

Evaluation for 1D-CNN-LSTM with Window Size: 100
Accuracy: 0.66